# setup

In [1]:
import torch
import os
import matplotlib.pyplot as plt

from utils import Imagenet_xy, train, eval, model_select

from tqdm import tqdm
from collections import OrderedDict

In [2]:
BATCH_SIZE = 512

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

# data

In [4]:
val = Imagenet_xy('val')

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/52 [00:00<?, ?it/s]

In [5]:
val_loader = torch.utils.data.DataLoader(val, batch_size = BATCH_SIZE)

val.__len__() / BATCH_SIZE

97.65625

# model

In [6]:
model_label = 'partial_rn18rn34_finetuned'
model = torch.load('output/{}.pt'.format(model_label))

# model_label = 'rn18'
# model = model_select(model_label)

In [7]:
model = model.to(device)

# eval

In [8]:
total = 0
correct = 0

model.eval()
with torch.no_grad():
    for i, sample in tqdm(enumerate(val_loader)):
        x, y = sample
        x = x.to(device)
        y = y.to(device)

        yhat = model.forward(x)

        _, predicted = torch.max(yhat.data, 1)
        total += y.size(0)
        correct += (predicted == y).sum().item()
1 - correct / total

0it [00:00, ?it/s]/home/josegfer/miniconda3/envs/mirror/lib/python3.8/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
98it [02:47,  1.70s/it]


0.42457999999999996

In [9]:
yhat

tensor([[-15.9284, -14.4794, -16.2697,  ...,  -2.7360, -11.0378, -15.4885],
        [-17.9174, -20.2324, -17.2837,  ...,  -3.7870, -13.8971, -14.4205],
        [-20.6090, -21.1157, -20.6883,  ...,  -1.4211, -15.9686, -18.2446],
        ...,
        [-11.8242,  -8.6278, -12.2542,  ...,  -8.8325,  -7.2751,  -6.8485],
        [-16.6109, -12.5530, -13.6192,  ..., -12.8579,  -4.9526,  -5.3527],
        [-19.7281, -16.4274, -15.5010,  ..., -12.1622, -14.1670,  -2.3129]],
       device='cuda:0')